<a href="https://colab.research.google.com/github/kim-nam-keyong/Data-Science-Analytic/blob/master/titanic_tfdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
import tensorflow_decision_forests as tfdf

In [4]:
print(tf.__version__)

2.15.0


In [10]:
#pip install tensorflow_decision_forests --upgrade

In [11]:
print(f"Found TF-DF {tfdf.__version__}")

Found TF-DF 1.8.1


In [17]:
df_train = pd.read_csv("/content/drive/MyDrive/Kaggle Data/titanic/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Kaggle Data/titanic/test.csv")

In [18]:
df_train.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [31]:
df_train.info() # object_cols = ['Name','Sex','Ticket','Cabin','Embarked'] 5개밖에 없으니 그냥 적자


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [80]:
a = df_train.shape
b= df_test.shape
print('train: {} test:{}'.format(a,b))
print(df_test.columns)

train: (891, 12) test:(418, 11)
Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [53]:
a = df_train.isnull().sum()
b=  df_test.isnull().sum()
print('train: {} \n test:{}'.format(a,b))

train: PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64 
 test:PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [43]:
object_cols = ['Name','Sex','Ticket','Cabin','Embarked']

In [30]:
a = df_train[object_cols].isnull().sum()
b = df_test[object_cols].isnull().sum()

print(a,b)
# train = Cabin 687, Embarked 2
# test = Cabin 327   is null sum

Name          0
Sex           0
Ticket        0
Cabin       687
Embarked      2
dtype: int64 Name          0
Sex           0
Ticket        0
Cabin       327
Embarked      0
dtype: int64


In [58]:
df_train[object_cols].head(10)  # , . 등 특수문자 들어간 것으로 확인  정제 필요
# 1. name 안에 특수 기호
# 2. ticket 에 앞에 알파벳, 띄어쓰기 후에 숫자  두가지 컬럼으로 분류해서 추가하기
#

,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,NaN,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,NaN,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,NaN,S
5,"Moran, Mr. James",male,330877,NaN,Q
6,"McCarthy, Mr. Timothy J",male,17463,E46,S
7,"Palsson, Master. Gosta Leonard",male,349909,NaN,S
8,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,347742,NaN,S
9,"Nasser, Mrs. Nicholas (Adele Achem)",female,237736,NaN,C


In [79]:
# 정제작업 정규화!
def preprocess(df):
    df = df.copy()   # 복사본 필수 되도록 원본은 건드리지 말 것

    def normalize_name(x):
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])
  # 특수기호 제거
    def ticket_number(x):
        return x.split(" ")[-1]
  # 띄어쓰기 기준으로 number가 적혀있으니 뒤에 번호만 빼온다

    def ticket_item(x):
        items = x.split(" ")
        if len(items) ==1:
            return None   # 숫자만 있는 것도 있으니 띄어쓰기 기준으로 하나의 리스트로 정렬 후 숫자만 있으면 len이 1이나오니까 무시
        return "_".join(items[0:-1]) # 마지막은 숫자만 있으니까 숫자를 제외한 모든 문자열을 반환

    df['Name'] = df['Name'].apply(normalize_name)
    df['Ticket_number'] = df['Ticket'].apply(ticket_number)
    df['Ticket_item'] = df['Ticket'].apply(ticket_item)
    return df

preprocessed_df_train = preprocess(df_train)
preprocessed_df_test = preprocess(df_test)



In [78]:
preprocessed_df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,1,0,3,Braund Mr. Owen Harris,male,22.0,1,0,A/5 21171,7.2500,NaN,S,21171,A/5
1,2,1,1,Cumings Mrs. John Bradley Florence Briggs Thayer,female,38.0,1,0,PC 17599,71.2833,C85,C,17599,PC
2,3,1,3,Heikkinen Miss. Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,3101282,STON/O2.
3,4,1,1,Futrelle Mrs. Jacques Heath Lily May Peel,female,35.0,1,0,113803,53.1000,C123,S,113803,None
4,5,0,3,Allen Mr. William Henry,male,35.0,0,0,373450,8.0500,NaN,S,373450,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,Montvila Rev. Juozas,male,27.0,0,0,211536,13.0000,NaN,S,211536,None
887,888,1,1,Graham Miss. Margaret Edith,female,19.0,0,0,112053,30.0000,B42,S,112053,None
888,889,0,3,Johnston Miss. Catherine Helen Carrie,female,NaN,1,2,W./C. 6607,23.4500,NaN,S,6607,W./C.
889,890,1,1,Behr Mr. Karl Howell,male,26.0,0,0,111369,30.0000,C148,C,111369,None


In [ ]:
# 기존에 있던 ticket , 전혀 상관없어 보이는 passengerId,/ survived 제거  합니다
def remove_values(list,values_to_remove):
  return [value for value in list if value not in values_to_remove]

value_to_remove = ['PassengerId','Ticket','Survived']

input_features = list(preprocessed_df_train.columns)
input_features_ = remove_values(input_features,value_to_remove)

print(f'input features:{input_features_}')

In [91]:
# pandas dataset -> Tensorflow dataset 변경

def tokenize_names(features, labels=None):
    features['Name'] = tf.strings.split(features['Name'])   # 문자열을 공백 기준으로 분할하여 토큰화
    return features, labels

ds_train = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_df_train,label="Survived").amp(tokenize_names)
ds_test =  tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_df_test).map(tokenize_names)

print(ds_train)

<_MapDataset element_spec=({'PassengerId': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Pclass': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Name': RaggedTensorSpec(TensorShape([None, None]), tf.string, 1, tf.int64), 'Sex': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Age': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'SibSp': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Parch': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Ticket': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Fare': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Cabin': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Embarked': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Ticket_number': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Ticket_item': TensorSpec(shape=(None,), dtype=tf.string, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


In [98]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0, # 로그 메시지 무시
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features_], # input_features_안에 있는 값들만 feature로 사용
    exclude_non_specified_features=True, # 다른것들은 feature로 사용하지 않는다.
    random_seed=1234   #
)
model.fit(ds_train)
self_evaluation = model.make_inspector().evaluation() # 모델의 성능평과 , 정확도와 손실 호출
print(f'Accuracy:{self_evaluation.accuracy} Loss:{self_evaluation.loss}')

Accuracy:0.804347813129425 Loss:0.8660705089569092


In [99]:
for features, labels in ds_train.take(1):
    print(features['Name'])
    if labels is not None:
        print(labels)
# 참고를 위한 tensor의 시각화

<tf.RaggedTensor [[b'Braund', b'Mr', b'Owen', b'Harris'],
 [b'Cumings', b'Mrs', b'John', b'Bradley', b'Florence', b'Briggs',
  b'Thayer']                                                      ,
 [b'Heikkinen', b'Miss', b'Laina'],
 [b'Futrelle', b'Mrs', b'Jacques', b'Heath', b'Lily', b'May', b'Peel'],
 [b'Allen', b'Mr', b'William', b'Henry'], [b'Moran', b'Mr', b'James'],
 [b'McCarthy', b'Mr', b'Timothy', b'J'],
 [b'Palsson', b'Master', b'Gosta', b'Leonard'],
 [b'Johnson', b'Mrs', b'Oscar', b'W', b'Elisabeth', b'Vilhelmina', b'Berg'],
 [b'Nasser', b'Mrs', b'Nicholas', b'Adele', b'Achem'],
 [b'Sandstrom', b'Miss', b'Marguerite', b'Rut'],
 [b'Bonnell', b'Miss', b'Elizabeth'],
 [b'Saundercock', b'Mr', b'William', b'Henry'],
 [b'Andersson', b'Mr', b'Anders', b'Johan'],
 [b'Vestrom', b'Miss', b'Hulda', b'Amanda', b'Adolfina'],
 [b'Hewlett', b'Mrs', b'Mary', b'D', b'Kingcome'],
 [b'Rice', b'Master', b'Eugene'],
 [b'Williams', b'Mr', b'Charles', b'Eugene'],
 [b'Vander', b'Planke', b'Mrs', b'Juli

In [94]:
model.summary()

Model: "gradient_boosted_trees_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (11):
	Age
	Cabin
	Embarked
	Fare
	Name
	Parch
	Pclass
	Sex
	SibSp
	Ticket_item
	Ticket_number

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.          "Name"  0.313988 ################
    2.           "Sex"  0.302357 ##############
    3.          "Fare"  0.262424 #########
    4.           "Age"  0.231696 ######
    5.        "Pclass"  0.220911 ####
    6.      "Embarked"  0.208079 ###
    7. "Ticket_number"  0.202060 ##
    8.   "Ticket_item"  0.195265 #
    9.         "SibSp"  0.184818 
   10.         "Parch"  0.180877 

Variable Importance: NUM_AS_ROOT:
    

In [91]:
def prediction_to_kaggle_format(model,threshold=0.5):
    proba_survive = model.predict(ds_test,verbose=0)[:,0]
    return pd.DataFrame({
        'PassengerID':df_test['PassengerId'],
        'Survived': (proba_survive >= threshold).astype(int)
    })

def make_submission(kaggle_predictions):
    path = '/content/sample_data'
    kaggle_predictions.to_csv(path,index=False)
    print(f'Submission exported to {path}')

kaggle_predictions = prediction_to_kaggle_format(model)
make_submission(kaggle_predictions)
!head /kaggle/working/submission.csv
#제출 폼